## Create Edge List
Creates an edge list for each patch in a provided patch raster and cost surface

In [1]:
#Import libraries
import arcpy
from arcpy.sa import *
import numpy as np
import pandas as pd
from skimage import graph
from matplotlib import pyplot as plt

In [2]:
#Set workspaces
arcpy.env.workspace = 'C:/Workspace/PronghornConnectivity/PronghornConnectivity.gdb'
arcpy.env.scratchWorkspace = 'C:/Workspace/PronghornConnectivity/Scratch/Scratch.gdb'
arcpy.env.overwriteOutput = True

In [3]:
def msg(txt):
    print(txt)
    arcpy.AddMessage(txt)
    return

In [4]:
#Get input datasets: Patches and CostSurface
patchRaster = 'C:/Workspace/PronghornConnectivity/PronghornConnectivity.gdb/Patches'
if not arcpy.Exists(patchRaster):
    print("Cannot locate patch raster")

costRaster = 'C:/Workspace/PronghornConnectivity/PronghornConnectivity.gdb/CostSurface'
if not arcpy.Exists(costRaster):
    print("Cannot locate cost surface raster")
    
maxCostDistance = 100000

outRaster = 'C:/Workspace/PronghornConnectivity/PronghornConnectivity.gdb/CD{}'

In [5]:
#Get the spatial reference, extent, and lower left coordinates
sr = arcpy.Describe(costRaster).spatialReference
cellSize = arcpy.Describe(costRaster).meanCellWidth
extent = arcpy.Describe(costRaster).extent
llCorner = arcpy.Point(extent.XMin,extent.YMin)

In [6]:
#Create arrays of the patch and core rasters
arrPatch = arcpy.RasterToNumPyArray(patchRaster,
                                    lower_left_corner=llCorner,
                                    nodata_to_value=-9999)

arrCost = arcpy.RasterToNumPyArray(costRaster,
                                   lower_left_corner=llCorner,
                                   nodata_to_value=-9999)

In [7]:
#Create a list of patchIDs
patchIDs = np.unique(arrPatch).tolist()
patchIDs.remove(-9999)

In [ ]:
arrList = []
for patchID in patchIDs[:5]:
    msg("{}...".format(patchID))

    #Reclassify cost in patch cells to zero
    arrCostMod = arrCost.copy()
    arrCostMod[arrPatch == patchID] = 0
    arrCostMod[arrCostMod == -9999] = 100000

    #Create the MCP object (Geometric accounts for diagonals)
    lg = graph.MCP_Geometric(arrCostMod, sampling=(cellSize, cellSize))

    #Get the index of a cell in the current patch ID
    i,j = np.where(arrPatch == patchID)
    startCell = i[0],j[0]
    startCells = list(zip(i,j))

    #Compute cost distances away from a source
    #lcd = lg.find_costs(starts=[startCell])[0]
    lcd = lg.find_costs(starts=startCells)[0]

    #Add array to arrList
    arrList.append(lcd)

#Save cost distance raster
#np.save("cd{}".format(patchID),lcd)

In [33]:
#Merge arrays
arrCDs = np.stack(arrList)
np.savez_compressed('costdists.npz',arrCDs)

In [27]:
arrCDs.shape

(5, 1628, 1731)

In [26]:
#Convert to raster
cdRaster = arcpy.NumPyArrayToRaster(arrCDs,llCorner,cellSize,cellSize)
cdRaster.save('C:/Workspace/PronghornConnectivity/PronghornConnectivity.gdb/CDs')

In [29]:
arcpy.da.Describe(cdRaster).keys()
arcpy.Describe(cdRaster).bandCount

1

In [ ]:
#Convert patch ID back to raster
lcdRaster = arcpy.NumPyArrayToRaster(lcd,llCorner,cellSize,cellSize)

lcdRaster.save(outRaster.format(patchID))
arcpy.DefineProjection_management(in_dataset=lcdRaster,coor_system=sr);

In [18]:
tb = lg.traceback([1,1])
tb[-10:]

[(4, 10),
 (4, 9),
 (4, 8),
 (4, 7),
 (4, 6),
 (4, 5),
 (4, 4),
 (3, 3),
 (2, 2),
 (1, 1)]